## OpenCL
Recently, there has been another revolution in computing power: the use of graphical processing units (GPUs) in general computation. For quite a while, GPUs have been used for heavy computation: rendering and drawing the complex shapes that appear on your monitor. As computer graphics became more sophisticated (animations, transitions, gaming), GPUs became more and more powerful. To handle the enourmous amount of computation involved in rendering a computer game (drawing millions of tiny polygons), GPUs began to add more and more processing units. While each processing unit ran at a small fraction of the speed of a CPU, many advanced GPUs included hundreds (or thousands!) of these units, giving GPUs (generally) more overall processing power than a CPU.

Instead of tasking these GPU units to draw shapes to the screen, we can instead use them for general computation using [OpenCL](http://www.khronos.org/opencl/). OpenCL is a programming framework for massively parallel computing using **any** computation device (CPU or GPU). While other GPU frameworks exist (most notably [nVidia's CUDA](http://www.nvidia.com/object/cuda_home_new.html)), OpenCL is an open standard that supports all types of CPUs and GPUs. Intel, nVidia and AMD are all partners of OpenCL, making the technology compatible with all their products.

In this post we will learn how to harness the massively-parallel architecture of your GPU for use in your Python programs.

### PyOpenCL Installation

The OpenCL Python module we will use is [PyOpenCL](http://mathema.tician.de/software/pyopencl/). Other options exist, but I have found this module to have great documentation, which is important when learning new techniques. There are several steps involved in installing this module:

* PyOpenCL requires the boost C++ libraries. You can install them via homebrew: ``brew install boost``
* Install the Python module: ``pip install pyopencl``

These steps *should* complete without any issues. Once everything is installed, re-open your terminal window and start python. Test if the module is installed by typing:

In [1]:
import pyopencl

### OpenCL Algorithms

For the most part, programs leveraging the OpenCL framework are similar to those using python-native functions: You want to **write your heavy computation into a function that will be called many times via a loop**. However, the code to create this function, and the "job server" that handles it, is much different. Before we delve into the code, let's go over how OpenCL moves your computations and data around:

1. The function that will handle the parallel computation (called a *kernel*) is compiled on the computing device. This computing device may be different than your primary CPU.
2. Your data is imported into your main Python program, and resides in system memory.
3. Your computing device, if it is not your main CPU, *cannot* access system memory. Before the kernel can be run, we have to copy the necessary data to the device's memory.
4. Your computing device runs the kernel, crunching the numbers and producing results. These results reside in the device memory.
5. To access the results in your main Python code, we must copy the results from device memory to system memory.

Before each run of the kernel we will have to copy over all necessary data, and after each run we will have to copy back the results. For most basic programs you won't have to think hard about this data transfer. For more advanced programs, however, there are two opposing concepts to consider:

* Data transfer takes time. While you will still gain massive improvements over a serial code, many copies to and from device memory can add significant overhead.
* Most GPUs have limited memory. My MacBook Pro's graphics memory is only 256MB, miniscule in the face of the 8GB of system memory, and most of that memory is already absorbed by running the display. This means that transfers of large arrays of data may fail due to insufficient memory size.

You will have to balance these two issues when deisgning a program. If you need to run a parallel analysis on a large array, do you copy the entire thing over to the GPU at once, or do you break it down into smaller pieces? Sometimes you will have to experiment with how much data your specific device can handle, and modify your algorithms appropriately.

### OpenCL Kernels

The OpenCL kernel is the main program that runs the parallel computation. As OpenCL is a fairly low-level framework, it is mostly written for the C and FORTRAN programming languages, not Python. As such, the PyOpenCL kernel **must be written in C**. This is a deviation from everything we have learned previously in the course, but only a slight one. As programming languages essentially work the same way, and therefore it will be easy to translate from Python into C.

There are many quirks and differences between writing a Python subroutine and an OpenCL kernel, but the best way to learn is by looking at already-written programs and discussing. Below are two examples of PyOpenCL kernels, and we will walk through the main points of each.

**Kernel Example 1: Vector Addition:** The kernel below takes in two vectors and adds them together.

In [2]:
_ = '''
__kernel void vadd(__global float* a, __global float* b, __global float* c, const unsigned int count)
{
    int i = get_global_id(0);
    if (i < count)
        c[i] = a[i] + b[i];
}
'''

Let's go over some of the main structure here:

1. Every kernel will start with ``__kernel void``. This is because **kernels do not return any values**. You will notice that vector ``c`` is the resulting vector, and it is used as an input into the function. This is true for all OpenCL kernels, and you will have to create an empty vector to save your calculation results to when you write your own OpenCL kernels.
2. Vector inputs are specified as ``__global [datatype]*``, while scalar inputs are specified as ``const [datatype]``.
3. Parallel computations are called multiple times via a loop. ``get_global_id(0)`` gets the counter from the loop. In OpenCL, it is possible to have a multi-dimensional loop, and each loop index is pulled from the vector ``get_global_id``. So programs with a 3D loop may have ``get_global_id(0)``, ``get_global_id(1)``, and ``get_global_id(2)``.
4. You may be wondering why we are passing the length of the vectors ``count`` to the kernel. This is because **C does not have a built-in way of finding the length of vectors**. If you have a loop over a vector, it is best to either hard-code the vector length into the kernel (if it will always be the same), or pass the length as a variable, as is done here.

**Kernel Example 2: Matrix Multiplication:** The kernel below takes in two matrices and multiplies them together.

In [3]:
_ = '''
__kernel void mmul(const int Mdim, const int Ndim, const int Pdim, __global float* A, __global float* B, __global float* C)
{
	int k;
	int i = get_global_id(0);
	int j = get_global_id(1);
	float tmp;
	if ((i < Ndim) && (j < Mdim))
	{
		tmp = 0.0;
		for (k = 0; k < Pdim; k++)
			tmp += A[i*Ndim+k] * B[k*Pdim+j];
		C[i*Ndim+j] = tmp;
	}
}
'''

We see all of the concepts mentioned in the vector addition case here as well: matrix dimensions are passed as variables into the code, and the counters from this 2D loop are pulled from ``get_global_id``. The only thing new may be the treatment of the matrices themselves. In Python, you would specify a matrix element via ``A[i,k]``, but here there is only one index. This is because **C cannot handle multi-dimensional arrays**. When passing a 2D+ array into an OpenCL kernel, you will have to transform it to a 1D vector, as is done here.

The last thing you may need to know about OpenCL kernels is that advanced (and even basic) math functions may not be available. If you have done any basic C or C++ programming, you know that most often you need to import the ``math.h`` or ``<cmath>`` headers to get access to most math functions beyond simple addition and multiplication. In OpenCL versions 1.1 and below, this header is not supported, and you lose out on all of these functions. In OpenCL 1.2+, however, ``math.h`` is imoported by default, and you can use any of the functions available there.

### PyOpenCL Syntax

Now that we know how to write our OpenCL kernels, we need to merge them into our Python programs. There are lots of parts to this, so the easiest thing to do is look at an example. Let's look at the full Python routine for the vector addition example:

In [3]:
# Import modules
import pyopencl as cl
import numpy as np

# Specify kernel
kernelsource = """
__kernel void vadd(
    __global float* a,
    __global float* b,
    __global float* c,
    const unsigned int count)
{
    int i = get_global_id(0);
    if (i < count)
        c[i] = a[i] + b[i];
}
"""

# Set-Up OpenCL environment
context = cl.create_some_context()                   # Choose Device
queue = cl.CommandQueue(context)                     # Create Command Queue
program = cl.Program(context, kernelsource).build()  # Build Kernel
vadd = program.vadd                                  # Build Program
vadd.set_scalar_arg_dtypes([None, None, None, np.uint32])  # Set program variable types

# Create input vectors
h_a = np.random.rand(100).astype(np.float32)
h_b = np.random.rand(100).astype(np.float32)
# Copy input vectors to device
d_a = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_a)
d_b = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_b)

# Create output vector
h_c = np.zeros(100).astype(np.float32)
# Copy output vector to device
d_c = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_c.nbytes)

# Execute kernel
vadd(queue, h_a.shape, None, d_a, d_b, d_c, 100)

# Read back results from the compute device
cl.enqueue_copy(queue, h_c, d_c)

h_c

array([ 1.95341969,  1.0749675 ,  1.02363062,  0.83763766,  0.77136636,
        1.11196077,  0.75834322,  1.08344018,  0.86464638,  0.71710968,
        1.74478376,  0.56969404,  0.4927811 ,  1.11953461,  1.21910572,
        0.77456665,  1.04502797,  1.49689388,  0.83661848,  1.26157629,
        0.84894246,  1.23585081,  1.3727479 ,  1.13614559,  1.53942013,
        0.25364208,  0.59682941,  1.27365041,  0.76266575,  0.50780398,
        0.84237862,  1.40266323,  1.31092596,  0.961272  ,  1.51455188,
        0.60342509,  1.78912687,  0.7039203 ,  0.66691148,  0.83449465,
        0.73041582,  1.78259635,  1.11679316,  1.04288459,  0.73431671,
        0.76272905,  0.97013021,  0.10226849,  1.21516252,  0.5592804 ,
        0.79124743,  0.64513403,  0.69029027,  0.93785429,  0.93938446,
        1.61494005,  0.93005008,  0.8206327 ,  1.60221291,  0.43116963,
        1.35946679,  0.58356392,  0.73393077,  1.18942904,  1.55843925,
        1.85044611,  1.29299426,  0.86520618,  0.58911562,  1.26

Let's walk through each of the comments in the above code.

#### Setting up the OpenCL environment:

* **Import modules**: At minimum you must import PyOpenCL and NumPy into your program.
* **Specify kernel**: This is the same kernel that we saw in the example before, but enclosed in triple quotations (""") and saved as a string variable. This string will be passed to PyOpenCL later in the code and built on the computing device.
* **Choose device**: The ``create_some_context()`` command will set up a computing environment on the specified computing device. If a machine has more than one possible options (a CPU and GPU), the user will be asked which device they wish to use.
* **Create command queue**: Similar to PP, PyOpenCL creates a command queue which passes loop iterations to idle cores on the computing device.
* **Build kernel**: The ``cl.Program`` command takes the computing environment and compiles the kernel on it.
* **Build program**: The kernel string may specify more than one program (you may have subroutines for the kernel. This line specifies which function within the kernel string should be fed the input values.
* **Set program variable types**: While you have specified the expected data types in the kernel, PyOpenCL must know how to format the input data so that everything matches up between devices. Vector inputs have a variable type of ``None``, while scalar data types are specified using the NumPy variable classifications. (In this example, we are passing an unsigned 32-bit integer).

#### Creating and copying data:

* **Create input vectors**: This is mostly stuff you have seen before. We create a vector of random numbers (although we could import the data from anywhere) to be input into the program. The only difference is the ``.astype(np.float32)``. This is because OpenCL (at least earlier versions of it) cannot handle the standard 64-bit floating point numbers that NumPy uses by default. If we passed a 64-bit float array to the OpenCL kernel, the memory locations would not line up, and we would get garbage results. Make sure to convert any arrays you are passing to OpenCL to 32-bit variables (float, int, uint, or otherwise).
* **Copy input vectors to device**: As stated in 10.3.2, the computing device cannot access system memory, and variables must be copied to the computing device before running the kernel. The variable ``hostbuf`` is the input vector that was generated in Python, and the output is the new device array. You will notice two memory flags are set for input vectors: ``READ_ONLY`` and ``COPY_HOST_PTR``. ``READ_ONLY`` means that the vector is an input value and should not be changed. ``COPY_HOST_PTR`` means that we want to copy out the memory reference position of the vector to the computing device. Both of these memory flags should be set for any vector being copied to the computing device
* **Create output vector**: We need to make a placeholder for the output vector we expect. Here we simply specify a ``np.zeros`` of the same length as the inputs, making sure to convert it to a ``np.float32`` type as well.
* **Copy output vector to device**: You will notice that the memory flags for this command are ``WRITE_ONLY``, which makes sense considering we want to be able to modify this array when printing results. The ``cl.Buffer`` call is slightly different in the fact that we pass it the number of bytes that ``h_c`` takes up, as opposed to passing the pointer (like we did for the inputs). This is because we are simply *creating the space for the results in the computing device's memory*, not really copying anything there.

#### Executing the kernel and returning results:

* **Execute kernel**: Now that everything is copied to the computing device, we can run the kernel on all the data. The ``vadd()`` command takes several arguments:
 * The first argument of the ``vadd()`` command is the command queue.
 * The second is the "shape" of the loop that should be run. Remmber, we get the increment of the loop within the kernel via the ``get_global_id`` function; the second argument specifies that array. In the example, we are passing the "shape" of our 1D input vector, meaning that we will have a 1D loop of length 1000.
 * The third argument is the data type of the kernel's return values. As kernels will not return data, this is always ``None``.
 * The last arguments are the input values into the ``vadd`` kernel. Remember, we need to pass the *copied* vector inputs and outputs to the program for it to work properly. Scalar values do not have to be copied, and can be specified using their current names in the Python program.
* **Copying results**: Once the kernel is complete, the results vector still resides on the computing device. To copy it back, we use the ``cl.enqueue_copy`` command to move the device's results (``d_c``) back to the Python program's vector (``h_c``).

After copying back the results from the kernel, your Python program can continue as normal (or print them out, as done aboe).

To help get a further grip on writing OpenCL programs, let's also look over the matrix multiplication example.

In [17]:
import pyopencl as cl
import numpy as np

# Specify kernel
kernelsource = """
__kernel void mmul(
    const int Mdim,
    const int Ndim,
    const int Pdim,
    __global float* A,
    __global float* B,
    __global float* C)
{
    int k;
    int i = get_global_id(0);
    int j = get_global_id(1);
    float tmp;
    if ((i < Ndim) && (j < Mdim)) {
        tmp = 0.0;
        for (k = 0; k < Pdim; k++)
            tmp += A[i*Ndim+k] * B[k*Mdim+j];
        C[i*Mdim+j] = tmp;
    }
}
"""

# Set-Up OpenCL environment
context = cl.create_some_context()                   # Choose Device
queue = cl.CommandQueue(context)                     # Create Command Queue
program = cl.Program(context, kernelsource).build()  # Build Kernel
mmul = program.mmul                                  # Build Program
mmul.set_scalar_arg_dtypes([np.uint32, np.uint32, np.uint32, None, None, None])  # Set program variable types

# Set up matrix sizes
Ndim, Pdim, Mdim = 500, 600, 700
sizeA = Ndim * Pdim
sizeB = Pdim * Mdim
sizeC = Ndim * Mdim

# Create input matrices
h_a = np.random.rand(sizeA).astype(np.float32)
h_b = np.random.rand(sizeB).astype(np.float32)
# Copy input matrices to device
d_a = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_a)
d_b = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=h_b)

# Create output matrix
h_c = np.zeros(sizeC).astype(np.float32)
# Copy output matrix to device
d_c = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, h_c.nbytes)

# Execute kernel
loopshape = (Ndim, Mdim)
mmul(queue, loopshape, None, Mdim, Ndim, Pdim, d_a, d_b, d_c)

# Read back results from the compute device
cl.enqueue_copy(queue, h_c, d_c)

np.reshape(h_c, (Ndim, Mdim))

array([[ 150.93429565,  151.26486206,  152.15020752, ...,  147.43330383,
         149.49533081,  152.80351257],
       [ 153.1399231 ,  151.36068726,  155.5138855 , ...,  146.41668701,
         152.52696228,  156.67251587],
       [ 148.86749268,  150.17826843,  147.72998047, ...,  142.72929382,
         149.19018555,  155.52618408],
       ..., 
       [ 163.92628479,  160.3528595 ,  163.3830719 , ...,  158.5124054 ,
         160.72270203,  164.8132019 ],
       [ 149.52366638,  147.22966003,  150.66053772, ...,  144.29576111,
         150.28434753,  154.43119812],
       [ 159.17720032,  153.43807983,  157.72988892, ...,  149.6648407 ,
         156.59985352,  160.33921814]], dtype=float32)

Note that the matrices had to be coerced into a 1-dimensions format, as C does not understand matrices. The ``np.ravel`` and ``np.reshape`` functions will be your friend here.

These two examples cover much of what you need to know about how to write OpenCL programs. The clever part is figuring out which parts of the code can be parallelized, and how you can manipulate your algorithms to fit within the confines of the OpenCL limitations (no ``math.h`` header, no matrices, etc.).